In [25]:
# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import sklearn.decomposition
from sklearn.model_selection import train_test_split


In [26]:
# load data
def load_data(filepath):
    data = pd.read_csv(filepath)
    return data

df = load_data("./dataset/CLAMP_Train.csv")

In [38]:
df.head()

,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,e_csum,...,CheckSum,Subsystem,DllCharacteristics,SizeOfStackReserve,SizeOfStackCommit,SizeOfHeapReserve,SizeOfHeapCommit,LoaderFlags,NumberOfRvaAndSizes,class
0,23117,144,3,0,4,0,65535,0,184,0,...,65452,2,1024,1048576,4096,1048576,4096,0,16,0
1,23117,144,3,0,4,0,65535,0,184,0,...,0,2,0,1048576,4096,1048576,4096,0,16,1
2,23117,144,3,0,4,0,65535,0,184,0,...,119291,2,0,1048576,4096,1048576,4096,0,16,0
3,23117,144,3,0,4,0,65535,0,184,0,...,91168,3,320,262144,4096,1048576,4096,0,16,0
4,23117,144,3,0,4,0,65535,0,184,0,...,82656,2,32768,1048576,4096,1048576,4096,0,16,1


In [28]:
def find_dataset_statistics(dataset:pd.DataFrame,target_col:str) -> tuple[int,int,int,int,float]:

    #Total number of records
    n_records = dataset.shape[0]
    #Total number of columns 
    n_columns = len(dataset.columns)
    #Number of records where target is negative
    n_negative = dataset[target_col].value_counts()[0]
    #Number of records where where target is positive
    n_positive = dataset[target_col].value_counts()[1]
    # Percentage of instances of positive target value
    perc_positive =  (n_positive/n_records)*100

    return n_records,n_columns,n_negative,n_positive,perc_positive

In [33]:

def train_test_split(  dataset: pd.DataFrame,
                       target_col: str, 
                       test_size: float,
                       stratify: bool,
                       random_state: int) -> tuple[pd.DataFrame,pd.DataFrame,pd.Series,pd.Series]:
    
#    split the dataset into train and test datasets
    train, test = train_test_split(dataset, test_size=test_size, random_state=random_state, stratify=dataset[target_col] if stratify else None )

    train_features = train.drop(target_col, axis=1)
    test_features = test.drop(target_col, axis=1)
    train_targets = train[target_col]
    test_targets = test[target_col]

    return train_features,test_features,train_targets,test_targets



(10, 5, 5, 5, 50.0)

In [ ]:
class PreprocessDataset:
    def __init__(self, 
                 train_features:pd.DataFrame, 
                 test_features:pd.DataFrame,
                 one_hot_encode_cols:list[str],
                 min_max_scale_cols:list[str],
                 n_components:int,
                 feature_engineering_functions:dict
                 ):
        # TODO: Add any state variables you may need to make your functions work
        self.one_hot_encode_cols = one_hot_encode_cols
        return

    def one_hot_encode_columns_train(self) -> pd.DataFrame:
        # TODO: Write the necessary code to create a dataframe with the categorical column names in 
        # the variable one_hot_encode_cols "one hot" encoded 
        ohe = OneHotEncoder()
        transformed = ohe.fit_transform(self.one_hot_encode_cols)
        one_hot_encoded_dataset = pd.DataFrame()

        return one_hot_encoded_dataset

    def one_hot_encode_columns_test(self) -> pd.DataFrame:
        # TODO: Write the necessary code to create a dataframe with the categorical column names in 
        # the variable one_hot_encode_cols "one hot" encoded 
        one_hot_encoded_dataset = pd.DataFrame()

        return one_hot_encoded_dataset

    def min_max_scaled_columns_train(self) -> pd.DataFrame:
        # TODO: Write the necessary code to create a dataframe with the numerical column names in 
        # the variable min_max_scale_cols scaled to the min and max of each column 
        min_max_scaled_dataset = pd.DataFrame()
        return min_max_scaled_dataset

    def min_max_scaled_columns_test(self) -> pd.DataFrame:
        # TODO: Write the necessary code to create a dataframe with the numerical column names in 
        # the variable min_max_scale_cols scaled to the min and max of each column 
        min_max_scaled_dataset = pd.DataFrame()
        return min_max_scaled_dataset
    
    def pca_train(self) -> pd.DataFrame:
        # TODO: use PCA to reduce the train_df to n_components principal components
        # Name your new columns component_1, component_2 .. component_n 
        pca_dataset = pd.DataFrame()
        return pca_dataset

    def pca_test(self) -> pd.DataFrame:
        # TODO: use PCA to reduce the test_df to n_components principal components
        # Name your new columns component_1, component_2 .. component_n 
        pca_dataset = pd.DataFrame()
        return pca_dataset

    def feature_engineering_train(self) -> pd.DataFrame:
        # TODO: Write the necessary code to create a dataframe with feature engineering functions applied 
        # from the feature_engineering_functions dict (the dict format is {'feature_name':function,})
        # each feature engineering function will take in type pd.DataFrame and return a pd.Series
        feature_engineered_dataset = pd.DataFrame()
        return feature_engineered_dataset

    def feature_engineering_test(self) -> pd.DataFrame:
        # TODO: Write the necessary code to create a dataframe with feature engineering functions applied 
        # from the feature_engineering_functions dict (the dict format is {'feature_name':function,})
        # each feature engineering function will take in type pd.DataFrame and return a pd.Series
        feature_engineered_dataset = pd.DataFrame()
        return feature_engineered_dataset

    def preprocess(self) -> tuple[pd.DataFrame,pd.DataFrame]:
        # TODO: Use the functions you wrote above to create train/test splits of the features and target with scaled and encoded values 
        # for the columns specified in the init function
        train_features = pd.DataFrame()
        test_features = pd.DataFrame()
        return train_features,test_features